In [283]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np
import math

# Exploration raw data

In [2]:
df=pd.read_json("/Users/David/code/Clementinelalande/marbotic/raw_data/export_23-5_minified.json")

In [79]:
df.shape

(1320229, 16)

In [80]:
df.columns

Index(['city', 'client_event_time', 'country', 'device_model', 'event_id',
       'event_type', 'is_attribution_event', 'language', 'os_version',
       'region', 'session_id', 'start_version', 'user_creation_time',
       'user_id', 'user_properties', 'event_properties'],
      dtype='object')

In [81]:
df.event_type.unique()

array(['[Activity] Good Answer', '[Activity] Wrong Answer',
       '[Activity] Stop', '[Activity] Start', '[Action] Click',
       '[Scene] Leave', '[Scene] Enter', '[Game] Pause',
       '[Scaffolding] Scaffolding', '[Game] Start', '[Error] Error',
       '[Game] Stop', '[OB] CommPreview', '[OB] DoYouHaveKit',
       '[OB] HasKitYes', '[OB] Email', '[OB] VerifCode',
       '[Activation] Game', '[OB] HasKitNo', '[OB] DiscoverTrial',
       '[OB] ClickTrial', '[Account] Sign In', '[Profile] Create',
       '[Profile] Update', '[OB] ProductSelection', '[OB] ProductHelp',
       '[OB] Type', '[Account] Sign Up', '[OB] ProfileName',
       '[OB] ProfileBirthdate', '[Activation] New Activation',
       '[Activation] Product', '[Activation] Piece', '[OB] Success',
       '[OB] ProductActivation', '[Scaffolding] Reset', '[OB] TrialStart',
       '[Account] Update Request', '[Profile] Delete', '[Popup] Open',
       '[Popup] Close', '[Account] Update Success', '[Account] Sign Out',
       '[Pr

In [82]:
df.head()

,city,client_event_time,country,device_model,event_id,event_type,is_attribution_event,language,os_version,region,session_id,start_version,user_creation_time,user_id,user_properties,event_properties
0,Dallas,2022-03-01 00:57:25.903,United States,"iPad13,4",1110,[Activity] Good Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531,"{'Newsletter': 'False', 'Products': ['Smart Le...","{'[Activity] Name': 'LinePanties', '[Activity]..."
1,Dallas,2022-03-01 00:57:41.878,United States,"iPad13,4",1111,[Activity] Wrong Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531,"{'Newsletter': 'False', 'Products': ['Smart Le...","{'[Activity] Name': 'LinePanties', '[Activity]..."
2,Toronto,2022-03-01 00:57:28.175,Canada,"iPad12,1",1430,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.2', '[Game] Code': 'Mar..."
3,Toronto,2022-03-01 00:57:28.177,Canada,"iPad12,1",1431,[Activity] Start,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.3', '[Game] Code': 'Mar..."
4,Toronto,2022-03-01 00:57:38.278,Canada,"iPad12,1",1432,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.3', '[Game] Code': 'Mar..."


In [109]:
df.event_properties[4]

{'[Activity] Name': '1.3',
 '[Game] Code': 'Marbotic',
 '[Time] Spent': 10,
 '[Scene] Name': 'Counting_Home',
 '[Profile] Age (days)': 2423,
 '[Game] Language': 'en_US',
 '[Activity] Solved': False,
 '[Scaffolding] Level': 11,
 '[Scene] Section': 'UP100'}

In [130]:
df.user_properties[4]

{'Newsletter': 'True',
 'Products': ['Smart Letters', 'Smart Numbers'],
 'Pieces': ['Uppercase Letters', 'Numbers'],
 'Language': 'English',
 'Type': 'Parent',
 'Country': 'CA',
 'Games': ['Marbotic Numbers',
  'Marbotic Letters',
  'Alphamonster',
  'Vocabubble',
  'Bla Bla Box',
  'Lil Reader',
  'Up To 100',
  '10 Fingers',
  'More Or Less',
  'Marbotic']}

In [83]:
user_properties_df=df.copy()

In [84]:
user_properties=user_properties_df[["user_id","user_properties"]]

# Import Dataframe with a query

## Connexcion à Google Query

In [85]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=PoQsXFrUUsLui0bniQSz9njix6vkmV&access_type=offline&code_challenge=S3iaAqXEe7U5N46YKdeJCKShEw-o-S2tI7TMNTfuXWM&code_challenge_method=S256


You are now logged in as [david.periers@gmail.com].
Your current project is [marbotic].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [86]:
!gcloud projects list

PROJECT_ID              NAME              PROJECT_NUMBER
marbotic                Marbotic          566474310042
symmetric-curve-346916  My First Project  175516577791
wagon-bootcamp-346916   Wagon bootcamp    697736453840


In [87]:
!gcloud config set project marbotic

Updated property [core/project].


In [88]:
!bq ls

/usr/local/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
     datasetId      
 ------------------ 
  marbotic_dataset  


In [89]:
!cat credentials

cat: credentials: No such file or directory


## Fonction extract with a query

In [264]:
from google.oauth2 import service_account

def extract():
    # Construct a BigQuery client object.

    key_path = "/Users/David/code/dperiers/GITHUB_NICKNAME/gcp/marbotic-587600c2c13f.json"

    credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    
    client = bigquery.Client(project="marbotic", credentials=credentials)

    query = """
        SELECT *
        FROM `marbotic.marbotic_dataset.events`
      
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [265]:
extract()

,user_id,start_version,os_version,client_event_time,language,event_id,session_id,region,is_attribution_event,user_creation_time,device_model,event_type,country,city
0,69077.0,1.0.8,10.3.3,2022-03-01 18:11:29.219000+00:00,English,7,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Account] Sign In,Canada,North Vancouver
1,69077.0,1.0.8,10.3.3,2022-03-01 18:20:11.873000+00:00,English,29,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductSelection,Canada,North Vancouver
2,69077.0,1.0.8,10.3.3,2022-03-01 18:20:19.948000+00:00,English,30,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductHelp,Canada,North Vancouver
3,69077.0,1.0.8,10.3.3,2022-03-01 18:20:22.647000+00:00,English,31,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductActivation,Canada,North Vancouver
4,69077.0,1.0.8,10.3.3,2022-03-01 18:21:22.139000+00:00,English,33,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Scaffolding] Reset,Canada,North Vancouver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,67216.0,1.0.7,14.7.1,2022-03-16 17:55:04.781000+00:00,English,579,1647453180177,Pennsylvania,False,2021-12-25 19:36:28.299000+00:00,iPad 6,[Scaffolding] Scaffolding,United States,Philadelphia
399996,67216.0,1.0.7,14.7.1,2022-03-16 17:59:22.978000+00:00,English,610,1647453180177,Pennsylvania,False,2021-12-25 19:36:28.299000+00:00,iPad 6,[Scaffolding] Scaffolding,United States,Philadelphia
399997,67216.0,1.0.7,14.7.1,2022-03-16 18:00:47.352000+00:00,English,622,1647453180177,Pennsylvania,False,2021-12-25 19:36:28.299000+00:00,iPad 6,[Scaffolding] Scaffolding,United States,Philadelphia
399998,67216.0,1.0.7,14.7.1,2022-04-22 20:49:12.107000+00:00,English,649,1650660458386,Pennsylvania,False,2021-12-25 19:36:28.299000+00:00,iPad 6,[Scaffolding] Scaffolding,United States,Philadelphia


# Exploration Marbotic User ID file

In [ ]:
test=pd.DataFrame(columns=["a","b"])
test

In [104]:
data=pd.read_csv("/Users/David/code/Clementinelalande/marbotic/raw_data/users_info_202206071551.csv",sep=";")

In [105]:
data.head()

,User id,Country,Language,News letter,Type,Attribution,Games,Products,Pieces,Purchases
0,59707,MX,English,True,Parent,PLU,"[""10 Fingers"",""Marbotic Letters"",""Marbotic Num...","[""Smart Kit"",""Smart Numbers"",""Nordic Kit""]","[""Nordic Letters"",""Numbers"",""Uppercase Letters""]",NaN
1,61748,NZ,English,False,Educator,ELU,"[""Marbotic Letters"",""Vocabubble"",""10 Fingers"",...","[""Smart Letters"",""Smart Numbers""]","[""Numbers"",""Uppercase Letters""]",NaN
2,54834,CA,English,True,Parent,PLU,"[""Sesame Street Numbers""]",NaN,NaN,NaN
3,69202,US,English,False,Parent,PLU,"[""Marbotic""]","[""Deluxe Learning Kit""]","[""Lowercase Letters"",""Numbers""]",NaN
4,67754,CA,English,True,Parent,PLU,"[""Marbotic""]","[""Smart Kit""]","[""Numbers"",""Uppercase Letters""]",NaN


In [111]:
data.shape

(13679, 10)

In [129]:
len(data["User id"].unique())

12858

# WAU table creation

## Load data

In [275]:
data=extract()

In [276]:
data.head(30)

,user_id,start_version,os_version,client_event_time,language,event_id,session_id,region,is_attribution_event,user_creation_time,device_model,event_type,country,city
0,69077.0,1.0.8,10.3.3,2022-03-01 18:11:29.219000+00:00,English,7,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Account] Sign In,Canada,North Vancouver
1,69077.0,1.0.8,10.3.3,2022-03-01 18:20:11.873000+00:00,English,29,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductSelection,Canada,North Vancouver
2,69077.0,1.0.8,10.3.3,2022-03-01 18:20:19.948000+00:00,English,30,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductHelp,Canada,North Vancouver
3,69077.0,1.0.8,10.3.3,2022-03-01 18:20:22.647000+00:00,English,31,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductActivation,Canada,North Vancouver
4,69077.0,1.0.8,10.3.3,2022-03-01 18:21:22.139000+00:00,English,33,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Scaffolding] Reset,Canada,North Vancouver
5,69077.0,1.0.8,10.3.3,2022-03-01 18:21:22.142000+00:00,English,34,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Scaffolding] Reset,Canada,North Vancouver
6,69077.0,1.0.8,10.3.3,2022-03-01 18:19:29.607000+00:00,English,25,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductSelection,Canada,North Vancouver
7,NaN,1.0.9,14.8.1,2022-03-03 17:02:18.287000+00:00,English,9,1646326831084,North Carolina,False,2022-03-03 17:00:31.072000+00:00,iPad 6,[OB] Type,United States,Winston-Salem
8,70846.0,1.0.9,14.8.1,2022-03-03 17:02:25.535000+00:00,English,10,1646326831084,North Carolina,False,2022-03-03 17:00:31.072000+00:00,iPad 6,[Account] Sign Up,United States,Winston-Salem
9,70846.0,1.0.9,14.8.1,2022-03-03 17:02:26.667000+00:00,English,11,1646326831084,North Carolina,False,2022-03-03 17:00:31.072000+00:00,iPad 6,[OB] ProfileName,United States,Winston-Salem


## Compute Weekly active users

In [162]:
import datetime
data=data[["user_id","client_event_time"]]
data["week2022"]=data["client_event_time"].map(lambda x: x.isocalendar()[1])
data.head()


,user_id,client_event_time,week2022
0,69077.0,2022-03-01 18:11:29.219000+00:00,9
1,69077.0,2022-03-01 18:20:11.873000+00:00,9
2,69077.0,2022-03-01 18:20:19.948000+00:00,9
3,69077.0,2022-03-01 18:20:22.647000+00:00,9
4,69077.0,2022-03-01 18:21:22.139000+00:00,9


In [164]:
data.head()

,user_id,client_event_time,week2022
0,69077.0,2022-03-01 18:11:29.219000+00:00,9
1,69077.0,2022-03-01 18:20:11.873000+00:00,9
2,69077.0,2022-03-01 18:20:19.948000+00:00,9
3,69077.0,2022-03-01 18:20:22.647000+00:00,9
4,69077.0,2022-03-01 18:21:22.139000+00:00,9


In [272]:
len(data.user_id.unique())

1482

In [151]:
start_timestamp=data["client_event_time"].min()
end_timestamp=data["client_event_time"].max()

In [157]:
start_timestamp.isocalendar()[1]

1

In [185]:
data.week2022.dtypes

dtype('int64')

In [227]:
categories=[(i) for i in data.week2022.unique()]
categories.sort()
categories=np.array(categories)

In [266]:
np.array(categories)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       22])

In [273]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

print(data.week2022.unique())  # Check unique values
print(end_timestamp.isocalendar()[1])
print(len(data.week2022.unique()))
#categories=[int(i) for i in range(1,end_timestamp.isocalendar()[1]+1)]

ohe = OneHotEncoder(sparse = False) # Instantiate encoder, put 0 if unknown category

ohe.fit(data[["week2022"]]) # Fit encoder

data_encoded = ohe.transform(data[['week2022']]) # Encode week number
data_encoded.shape
data_encoded=pd.DataFrame(data_encoded, columns=ohe.categories_)
wau=data[["user_id"]]
wau[data_encoded.columns.to_list()]=data_encoded[data_encoded.columns.to_list()]
wau=wau.groupby(by="user_id").sum()
wau

[ 9  5  8  6  7  3  4  2  1 22 13 12 11 10 16 14 15 17]
22
18


/Users/David/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,"(1,)","(2,)","(3,)","(4,)","(5,)","(6,)","(7,)","(8,)","(9,)","(10,)","(11,)","(12,)","(13,)","(14,)","(15,)","(16,)","(17,)","(22,)"
user_id,,,,,,,,,,,,,,,,,,
211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,224.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0
564.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,0.0,0.0
834.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
870.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,3.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72707.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,0.0,0.0
72709.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0
72711.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,124.0,0.0,0.0


## Compute maturity cohorts

In [309]:
cohorts=data[["user_id","client_event_time","user_creation_time"]]
cohorts["maturity"]=cohorts["client_event_time"]-cohorts["user_creation_time"]
cohorts["maturity_years"]=cohorts["maturity"].map(lambda x: math.floor(x.days/365))
cohorts["maturity_weeks"]=cohorts["maturity"].map(lambda x: math.floor(x.days/7))
cohorts["maturity_months"]=cohorts["maturity"].map(lambda x: math.floor(x.days/30))
cohorts["maturity_days"]=cohorts["maturity"].map(lambda x: x.days)

/var/folders/0b/xnxkv0n57js_989_n8cgs5y40000gn/T/ipykernel_10649/2029688389.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cohorts["maturity"]=cohorts["client_event_time"]-cohorts["user_creation_time"]
/var/folders/0b/xnxkv0n57js_989_n8cgs5y40000gn/T/ipykernel_10649/2029688389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cohorts["maturity_years"]=cohorts["maturity"].map(lambda x: math.floor(x.days/365))
/var/folders/0b/xnxkv0n57js_989_n8cgs5y40000gn/T/ipykernel_10649/2029688389.py:4: SettingWith

### Maturity table by days

In [310]:
cohorts["week2022"]=cohorts["client_event_time"].map(lambda x: x.isocalendar()[1])

ohe = OneHotEncoder(sparse = False) # Instantiate encoder, put 0 if unknown category

ohe.fit(cohorts[["week2022"]]) # Fit encoder

data_encoded = ohe.transform(cohorts[['week2022']]) # Encode week number
data_encoded=pd.DataFrame(data_encoded, columns=ohe.categories_)
days_maturity_table=cohorts[["user_id","maturity_days"]]
days_maturity_table[data_encoded.columns.to_list()]=data_encoded[data_encoded.columns.to_list()]
days_maturity_table=days_maturity_table.groupby(by="user_id").sum()
days_maturity_table=days_maturity_table.groupby(by="maturity_days").count()
days_maturity_table

/var/folders/0b/xnxkv0n57js_989_n8cgs5y40000gn/T/ipykernel_10649/1987203421.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cohorts["week2022"]=cohorts["client_event_time"].map(lambda x: x.isocalendar()[1])
/Users/David/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,"(1,)","(2,)","(3,)","(4,)","(5,)","(6,)","(7,)","(8,)","(9,)","(10,)","(11,)","(12,)","(13,)","(14,)","(15,)","(16,)","(17,)","(22,)"
maturity_days,,,,,,,,,,,,,,,,,,
-38,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165
1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728522,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
889052,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
950243,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


### Maturity table by weeks

In [304]:
cohorts["week2022"]=cohorts["client_event_time"].map(lambda x: x.isocalendar()[1])

ohe = OneHotEncoder(sparse = False) # Instantiate encoder, put 0 if unknown category

ohe.fit(cohorts[["week2022"]]) # Fit encoder

data_encoded = ohe.transform(cohorts[['week2022']]) # Encode week number
data_encoded=pd.DataFrame(data_encoded, columns=ohe.categories_)
days_maturity_table=cohorts[["maturity_days"]]
days_maturity_table[data_encoded.columns.to_list()]=data_encoded[data_encoded.columns.to_list()]
days_maturity_table=days_maturity_table.groupby(by="maturity_days").sum()
days_maturity_table

,user_id,client_event_time,user_creation_time,maturity,maturity_years,maturity_months,maturity_days,week2022
0,69077.0,2022-03-01 18:11:29.219000+00:00,2022-01-24 05:21:26.336000+00:00,36 days 12:50:02.883000,0,0,36,9
1,69077.0,2022-03-01 18:20:11.873000+00:00,2022-01-24 05:21:26.336000+00:00,36 days 12:58:45.537000,0,0,36,9
2,69077.0,2022-03-01 18:20:19.948000+00:00,2022-01-24 05:21:26.336000+00:00,36 days 12:58:53.612000,0,0,36,9
3,69077.0,2022-03-01 18:20:22.647000+00:00,2022-01-24 05:21:26.336000+00:00,36 days 12:58:56.311000,0,0,36,9
4,69077.0,2022-03-01 18:21:22.139000+00:00,2022-01-24 05:21:26.336000+00:00,36 days 12:59:55.803000,0,0,36,9
...,...,...,...,...,...,...,...,...
399995,67216.0,2022-03-16 17:55:04.781000+00:00,2021-12-25 19:36:28.299000+00:00,80 days 22:18:36.482000,0,1,80,11
399996,67216.0,2022-03-16 17:59:22.978000+00:00,2021-12-25 19:36:28.299000+00:00,80 days 22:22:54.679000,0,1,80,11
399997,67216.0,2022-03-16 18:00:47.352000+00:00,2021-12-25 19:36:28.299000+00:00,80 days 22:24:19.053000,0,1,80,11
399998,67216.0,2022-04-22 20:49:12.107000+00:00,2021-12-25 19:36:28.299000+00:00,118 days 01:12:43.808000,0,2,118,16
